>Instituto de Informática  <br>
Nome da/do estudante <br>
### Universidade Federal de Goiás <br>
### **Inteligência Artificial Aplicada a Saúde** <br>

# Câncer Cervical

#### **Lê o conjunto de dados e os estrutura em diferentes variáveis**

In [ ]:
#fazendo download do dataset, de maneira local 
!gdown --id 1L_tE_lrW-qSQK1FKBnol2VEJ1Yv8krsh # treinamento
!gdown --id 125xeEzDcAOKQWHWrXYG6-Lm5IOORoj5Y # teste
!gdown --id 1K_BxPAJMUMdkWhRx1D2gbnQxHOz5Kq34 # todo o df
#!gdown --id 1J-XUzcsIfbcAH3hiZHcTD1avQRgb6qt6 # ytest (uso opcional)

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1L_tE_lrW-qSQK1FKBnol2VEJ1Yv8krsh
To: /content/Treinamento.csv
100% 3.04k/3.04k [00:00<00:00, 9.55MB/s]
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=125xeEzDcAOKQWHWrXYG6-Lm5IOORoj5Y
To: /content/Teste.csv
100% 1.49k/1.49k [00:00<00:00, 3.65MB/s]
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Definir o caminho onde o dataset se encontra
path_train = '/content/Treinamento.csv'
path_test = '/content/Teste.csv'
#path_idx_normal = '/content/Dados.csv' #uso opcional  /// acho que e pra comparar ECG normales 

# Carregar dataset 
xtrain = pd.read_csv(path_train)
xtest = pd.read_csv(path_test)
#idx_normal = pd.read_csv(path_idx_normal)

# Separar y referencia
ytest = xtest['ca_cervix']
ytrain = xtrain['ca_cervix']

# Remover y do conjunto de treino
xtest = xtest.drop(columns=['ca_cervix'])
xtrain = xtrain.drop(columns=['ca_cervix'])

#### **Gera o modelo para realizar os agrupamentos**


> O número de clusters é igual à quantidade de classes no problema



In [ ]:
from sklearn.cluster import KMeans

# Gerar clusters pelo kmeans
km = KMeans(n_clusters=2, random_state=22).fit(xtrain)

#### **Identifica as classes mais próximas de cada centróide para realizar a classificação**
> Portanto se o cluster 0 estiver mais próximo do grupo de controle <br>
Então o cluster 0 representa o grupo de controle e o cluster 1 os paciêntes com cancêr cervical. 

In [ ]:
real_c_y = []
for idx, c in enumerate(km.cluster_centers_):
  dist = np.linalg.norm(xtrain - c, axis=1)
  real_c_y.append((idx, ytrain.iloc[np.where(dist == dist.min())[0][0]]))

  del dist

print(real_c_y)

[(0, 0), (1, 0)]


In [ ]:
dist = np.linalg.norm(xtrain - km.cluster_centers_[0], axis=1)
a = dist.min()
dist = np.linalg.norm(xtrain - km.cluster_centers_[1], axis=1)
b = dist.min()

print(a,b)

8.483599621893397 9.422805113040761


Cluster 0 - Paciêntes sem cancêr cervical <br>
Cluster 1 - Paciêntes com cancêr cervical 

In [ ]:
real_c_y = np.array(real_c_y)
real_c_y[1][1] = 1
real_c_y = tuple(real_c_y)

In [ ]:
real_c_y

(array([0, 0]), array([1, 1]))

#### **Atribui os novos dados aos clusters mais próximos**



In [ ]:
result = km.predict(xtest)
result_copy = result.copy()

In [ ]:
# Substituir os clusters formados pelo kmeans pelas classes reais
for c in range(len(real_c_y),):
  result[np.where(result_copy == c)[0]] = real_c_y[c][1]

In [ ]:
# Quantidade de Paciêntes sem cancêr cervical e Paciêntes com cancêr cervical com a classificação via km
unique, counts = np.unique(result, return_counts=True)
print(np.asarray((unique, counts)).T)

[[ 0 11]
 [ 1 10]]


#### **Gera a matriz de confusão e acurácia total da classificação**


> As linhas são a referência (y real)
>> Linha 1 = Paciêntes com cancêr cervical <br>
Linha 2 = Paciêntes sem cancêr cervical 


> As colunas correspondem a saida do modelo (y predito)
>> Coluna 1 = Paciêntes com cancêr cervical <br>
Coluna 2 = Paciêntes sem cancêr cervical


In [ ]:
def get_results(ytest, result):
  # Verdadeiro positivo: Paciêntes com cancêr cervical, modelo prediz Paciêntes com cancêr cervical
  # Verdadeiro negativo: Paciêntes sim cancêr cervical, modelo prediz Paciêntes sim cancêr cervical
  # Falso positivo: Paciêntes com cancêr cervical, modelo prediz Paciêntes sem cancêr cervical
  # Falso negativo: Paciêntes sem cancêr cervical, modelo prediz Paciêntes com cancêr cervical

  results = []
  for esperado, predito in zip(ytest, result):
    
    # o modelo acertou a predição
    if esperado == predito:
      if predito == 1: # a predição foi Paciêntes com cancêr cervical (positivo)
        result2 = "TP" # True Positive
    
      else: # a predição foi Paciêntes sem cancêr cervical (negativo)
        result2 = "TN" # True Negative
    
    # o modelo errou a predição
    else:
        if predito == 1: # a predição foi Paciêntes com cancêr cervical (positivo)
          result2 = "FP" # False Positive
    
        else: # a predição era Paciêntes sem cancêr cervical (negativo)
          result2 = "FN" # False Negative

    results.append(result2)
  return results


results = get_results(ytest.values, result)

In [ ]:
# obtendo os index das amostras que a prediçãod o modelo resultou em TP, TN, FP e FN
index_TP = [i for i, val in enumerate(results) if val == 'TP']
index_TN = [i for i, val in enumerate(results) if val == 'TN']
index_FP = [i for i, val in enumerate(results) if val == 'FP']
index_FN = [i for i, val in enumerate(results) if val == 'FN']

In [ ]:
from sklearn.metrics import confusion_matrix as conf_matrix

# Exibe a matriz de confusão
print(conf_matrix(ytest, result))

print("Verdadeiro positivo: ", len(index_TP))
print("Verdadeiro negativo: ", len(index_TN))
print("Falso positivo: ", len(index_FP))
print("Falso negativo: ", len(index_FN))

[[10  5]
 [ 1  5]]
Verdadeiro positivo:  5
Verdadeiro negativo:  10
Falso positivo:  5
Falso negativo:  1


In [ ]:
# Mostra acurácia total da classificação
correct_values = np.where(ytest == result)[0]
print("Total acuracy: %f" % (100*(correct_values.shape[0] / result.shape[0])) + " %")

del correct_values

Total acuracy: 71.428571 %


In [ ]:
result.shape[0]

21